In [1]:
%env PJRT_DEVICE=TPU
%env XLA_IR_DEBUG=1
%env XLA_HLO_DEBUG=1
%env TPU_LIBRARY_PATH=/workspaces/torch/_libtpu.so

# MaxText flags except that we disable optimization barrier removal: crashes in native code
# %env LIBTPU_INIT_ARGS=--xla_enable_async_all_gather=true --xla_tpu_enable_async_collective_fusion=true --xla_tpu_enable_async_collective_fusion_fuse_all_gather=true --xla_tpu_enable_async_collective_fusion_multiple_steps=true --xla_tpu_decompose_all_gather_einsum=true --xla_tpu_decompose_einsum_reduce_scatter=true --xla_tpu_scoped_vmem_limit_kib=98304 --xla_tpu_spmd_rng_bit_generator_unsafe=true --xla_tpu_overlap_compute_collective_tc=true --xla_tpu_use_enhanced_launch_barrier=true --xla_tpu_enable_all_experimental_scheduler_features=true --xla_tpu_enable_scheduler_memory_pressure_tracking=true --xla_tpu_host_transfer_overlap_limit=2 --xla_tpu_aggressive_opt_barrier_removal=DISABLED --xla_lhs_prioritize_async_depth_over_stall=ENABLED --xla_tpu_enable_ag_backward_pipelining=true --xla_should_allow_loop_variant_parameter_in_chain=ENABLED --xla_should_add_loop_invariant_op_in_chain=ENABLED --xla_max_concurrent_host_send_recv=100 --xla_tpu_scheduler_percent_shared_memory_limit=100 --xla_latency_hiding_scheduler_rerun=2

# Still crashes in native code
# %env LIBTPU_INIT_ARGS=--xla_enable_async_all_gather=true --xla_tpu_enable_async_collective_fusion=true --xla_tpu_enable_async_collective_fusion_fuse_all_gather=true --xla_tpu_enable_async_collective_fusion_multiple_steps=true --xla_tpu_decompose_all_gather_einsum=true --xla_tpu_decompose_einsum_reduce_scatter=true --xla_tpu_scoped_vmem_limit_kib=98304 --xla_tpu_spmd_rng_bit_generator_unsafe=true --xla_tpu_overlap_compute_collective_tc=true --xla_tpu_use_enhanced_launch_barrier=true --xla_tpu_enable_scheduler_memory_pressure_tracking=true --xla_tpu_host_transfer_overlap_limit=2 --xla_tpu_aggressive_opt_barrier_removal=DISABLED --xla_lhs_prioritize_async_depth_over_stall=ENABLED --xla_tpu_enable_ag_backward_pipelining=true --xla_should_allow_loop_variant_parameter_in_chain=ENABLED --xla_should_add_loop_invariant_op_in_chain=ENABLED --xla_max_concurrent_host_send_recv=100 --xla_tpu_scheduler_percent_shared_memory_limit=100

# Removed some more flags, and 1% scheduler shared memory limit: OK
%env LIBTPU_INIT_ARGS=--xla_tpu_overlap_compute_collective_tc=true --xla_tpu_use_enhanced_launch_barrier=true --xla_tpu_enable_scheduler_memory_pressure_tracking=true --xla_tpu_host_transfer_overlap_limit=2 --xla_tpu_aggressive_opt_barrier_removal=DISABLED --xla_lhs_prioritize_async_depth_over_stall=ENABLED --xla_tpu_enable_ag_backward_pipelining=true --xla_should_allow_loop_variant_parameter_in_chain=ENABLED --xla_should_add_loop_invariant_op_in_chain=ENABLED --xla_max_concurrent_host_send_recv=100 --xla_tpu_scheduler_percent_shared_memory_limit=1

env: PJRT_DEVICE=TPU
env: XLA_IR_DEBUG=1
env: XLA_HLO_DEBUG=1
env: TPU_LIBRARY_PATH=/workspaces/torch/_libtpu.so
env: LIBTPU_INIT_ARGS=--xla_tpu_overlap_compute_collective_tc=true --xla_tpu_use_enhanced_launch_barrier=true --xla_tpu_enable_scheduler_memory_pressure_tracking=true --xla_tpu_host_transfer_overlap_limit=2 --xla_tpu_aggressive_opt_barrier_removal=DISABLED --xla_lhs_prioritize_async_depth_over_stall=ENABLED --xla_tpu_enable_ag_backward_pipelining=true --xla_should_allow_loop_variant_parameter_in_chain=ENABLED --xla_should_add_loop_invariant_op_in_chain=ENABLED --xla_max_concurrent_host_send_recv=100 --xla_tpu_scheduler_percent_shared_memory_limit=1


In [2]:
import torch_xla
import torch
from torch_xla import runtime as xr

In [3]:
from torch.autograd.graph import saved_tensors_hooks
from torch_xla.experimental.stablehlo_custom_call import (
  place_to_host, place_to_device
)

class OffloadingModule(torch.nn.Module):
  def __init__(self, m):
    super().__init__()
    self.m = m

  def forward(self, *args, **kwargs):
    with saved_tensors_hooks(place_to_host, place_to_device):
      return self.m(*args, **kwargs)

In [4]:
import decoder_only_model
from trainer import TrainDecoderOnlyBase
import functools

## Test 2D FSDP+TP sharding

In [5]:
import torch
import numpy as np
import torch_xla.distributed.spmd as xs
import torch_xla.utils.utils as xu
import torch_xla.distributed.parallel_loader as pl
from torch_xla import runtime as xr
from itertools import chain

class TrainDecoderOnlyFSDPv2(TrainDecoderOnlyBase):

  def __init__(self):
    super().__init__(decoder_only_model.DecoderOnlyConfig(
       hidden_size=4096,
       num_hidden_layers=32,
       num_attention_heads=16,
       num_key_value_heads=8,
       intermediate_size=8192,
       vocab_size=16384,
    ))
    # Define the mesh following common SPMD practice
    num_devices = xr.global_runtime_device_count()
    tensor_axis = 4
    fsdp_axis = num_devices // tensor_axis
    mesh_shape = (fsdp_axis, tensor_axis)
    print(f"Single-slice sharding: mesh={mesh_shape}")
    spmd_mesh = xs.Mesh(list(range(num_devices)), mesh_shape, ('fsdp', 'tensor'))
    xs.set_global_mesh(spmd_mesh)

    model: decoder_only_model.DecoderOnlyModel = self.model  # type:ignore
    self.model = model
   
    # Mark model weights to be sharded
    for name, param in chain(model.named_parameters(), model.named_buffers()):
      print('> [2D] Sharding tensor', name, param.shape)

      # Here we intentionally skip layernorm and moe.gate weights given they are small.
      if 'embed_tokens' in name:
          xs.mark_sharding(param, spmd_mesh, ('fsdp', 'tensor'))
      elif 'q_proj' in name or 'k_proj' in name or 'v_proj' in name:
          xs.mark_sharding(param, spmd_mesh, ('tensor', 'fsdp'))
      elif 'o_proj' in name:
          xs.mark_sharding(param, spmd_mesh, ('fsdp', 'tensor'))
      elif 'gate_proj' in name or 'up_proj' in name:
          xs.mark_sharding(param, spmd_mesh, ('tensor', 'fsdp'))
      elif 'down_proj' in name:
          xs.mark_sharding(param, spmd_mesh, ('fsdp', 'tensor'))
      elif 'lm_head' in name:
          xs.mark_sharding(param, spmd_mesh, (('tensor', 'fsdp'), None))

      print(f'{name} {torch_xla._XLAC._get_xla_sharding_spec(param)}')

    # Shard the input.
    # Scale the batch size with num_devices since there will be only one
    # process that handles all runtime devices.
    self.batch_size *= num_devices
    train_loader = xu.SampleGenerator(
        data=(torch.randint(
            0,
            self.config.vocab_size, (self.batch_size, self.seq_len),
            dtype=torch.int64,
            device='cpu'),
              torch.randint(
                  0,
                  self.config.vocab_size, (self.batch_size, self.seq_len),
                  dtype=torch.int64,
                  device='cpu')),
        sample_count=self.train_dataset_len // self.batch_size)
    self.train_device_loader = pl.MpDeviceLoader(
        train_loader,
        self.device,
        # Shard the input's batch dimension along the `fsdp` axis, no sharding along other dimensions
        input_sharding=xs.ShardingSpec(spmd_mesh, ('fsdp', None)))  # type:ignore
    
    # Apply checkpoint to each DecoderLayer layer.
    from torch_xla.distributed.fsdp import checkpoint_module
    for i, block in enumerate(self.model.layers):
        self.model.layers[i] = checkpoint_module(block)
        
    # Apply offloading to each DecoderLayer layer.
    from torch_xla.distributed.fsdp import checkpoint_module
    for i, block in enumerate(self.model.layers):
        self.model.layers[i] = OffloadingModule(block)

    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.00001)
    torch_xla.sync(wait=True)


In [6]:
xr.use_spmd()
base = TrainDecoderOnlyFSDPv2()

Single-slice sharding: mesh=(2, 4)
> [2D] Sharding tensor embed_tokens.weight torch.Size([16384, 4096])
embed_tokens.weight {devices=[2,4]0,1,2,3,4,5,6,7}
> [2D] Sharding tensor layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
layers.0.self_attn.q_proj.weight {devices=[4,2]0,4,1,5,2,6,3,7}
> [2D] Sharding tensor layers.0.self_attn.k_proj.weight torch.Size([2048, 4096])
layers.0.self_attn.k_proj.weight {devices=[4,2]0,4,1,5,2,6,3,7}
> [2D] Sharding tensor layers.0.self_attn.v_proj.weight torch.Size([2048, 4096])
layers.0.self_attn.v_proj.weight {devices=[4,2]0,4,1,5,2,6,3,7}
> [2D] Sharding tensor layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
layers.0.self_attn.o_proj.weight {devices=[2,4]0,1,2,3,4,5,6,7}
> [2D] Sharding tensor layers.0.mlp.gate_proj.weight torch.Size([8192, 4096])
layers.0.mlp.gate_proj.weight {devices=[4,2]0,4,1,5,2,6,3,7}
> [2D] Sharding tensor layers.0.mlp.up_proj.weight torch.Size([8192, 4096])
layers.0.mlp.up_proj.weight {devices=[4,2]0,4,1,5,

In [7]:
print("Compiling model")
base.num_steps = 3
base.start_training()
torch_xla.sync(wait=True)

print("Profiling model")
import torch_xla.debug.profiler as xp
server = xp.start_server(9012)
xp.trace_detached(
    service_addr="localhost:9012", logdir="profile/", duration_ms=15000)
base.num_steps = 5
base.start_training()
torch_xla.sync(wait=True)
del server

Compiling model
Epoch 1 train begin  6:39AM UTC on Nov 10, 2024


/workspaces/torch/pytorch/xla/torch_xla/utils/checkpoint.py:183: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  torch.cuda.amp.autocast(**ctx.gpu_autocast_kwargs), \
/workspaces/torch/pytorch/xla/torch_xla/utils/checkpoint.py:184: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):


epoch: 1, step: 0, loss: 9.872329711914062, rate: 5.7734769669556405
epoch: 1, step: 1, loss: 9.872330665588379, rate: 8.71372877889234
Epoch 1 train end  6:41AM UTC on Nov 10, 2024
epoch: 1, step: 2, loss: 9.87232780456543, rate: 31.027428920451158
Profiling model
Epoch 1 train begin  6:41AM UTC on Nov 10, 2024
Starting to trace for 15000 ms. Remaining attempt(s): 2


2024-11-10 06:41:52.863990: W external/tsl/tsl/profiler/lib/profiler_session.cc:109] Profiling is late by 937820 nanoseconds and will start immediately.


epoch: 1, step: 0, loss: 9.87232494354248, rate: 44.762044265507
epoch: 1, step: 1, loss: 9.87232780456543, rate: 45.46384326382709
epoch: 1, step: 2, loss: 9.872328758239746, rate: 45.746723364910466
epoch: 1, step: 3, loss: 9.872325897216797, rate: 45.89120208792383
Epoch 1 train end  6:42AM UTC on Nov 10, 2024
epoch: 1, step: 4, loss: 9.872322082519531, rate: 45.94576950616121


In [8]:
from visualize import visualize_tensor_sharding
_ = visualize_tensor_sharding(base.model.embed_tokens.weight, use_color=False)

4096               
     ┌───────┬───────┬───────┬───────┐
     │       │       │       │       │
     │ TPU 0 │ TPU 1 │ TPU 2 │ TPU 3 │
     │       │       │       │       │
16384├───────┼───────┼───────┼───────┤
     │       │       │       │       │
     │ TPU 4 │ TPU 5 │ TPU 6 │ TPU 7 │
     │       │       │       │       │
     └───────┴───────┴───────┴───────┘

In [9]:
_ = visualize_tensor_sharding(base.model.layers[0].m.self_attn.q_proj.weight, use_color=False)

4096       
    ┌───────┬───────┐
    │       │       │
    │ TPU 0 │ TPU 4 │
    │       │       │
    ├───────┼───────┤
    │       │       │
    │ TPU 1 │ TPU 5 │
    │       │       │
4096├───────┼───────┤
    │       │       │
    │ TPU 2 │ TPU 6 │
    │       │       │
    ├───────┼───────┤
    │       │       │
    │ TPU 3 │ TPU 7 │
    │       │       │
    └───────┴───────┘

In [10]:
_ = visualize_tensor_sharding(base.model.layers[0].m.self_attn.o_proj.weight, use_color=False)

4096               
    ┌───────┬───────┬───────┬───────┐
    │       │       │       │       │
    │ TPU 0 │ TPU 1 │ TPU 2 │ TPU 3 │
    │       │       │       │       │
4096├───────┼───────┼───────┼───────┤
    │       │       │       │       │
    │ TPU 4 │ TPU 5 │ TPU 6 │ TPU 7 │
    │       │       │       │       │
    └───────┴───────┴───────┴───────┘